In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.nn.functional import normalize
import time
import pickle
from datetime import timedelta
np.random.seed(64)

#path = '/home/ahardy/ENSchallengeData/' 
def randomA(D=250, F=10):  
    
    M = np.random.randn(D,F)
    randomStiefel = np.linalg.qr(M)[0] # Apply Gram-Schmidt algorithm to the columns of M
    
    return randomStiefel



def checkOrthonormality(A): 
    
    bool = True
    D, F = A.shape   
    Error = pd.DataFrame(A.T @ A - np.eye(F)).abs()
    
    if any(Error.unstack() > 1e-6):
        bool = False
     
    return bool

In [5]:
X_train = pd.read_csv('./X_train.csv', index_col=0, sep=',')
X_train.columns.name = 'date'

Y_train = pd.read_csv('./Y_train.csv', index_col=0, sep=',')
Y_train.columns.name = 'date'

In [41]:
def My_Own_CF(n = 10 ):
    perm = np.random.permutation(range(50))
    valset = perm[:n]
    trainset = perm[n:]
    X_1 = X_train.iloc[valset,]
    X_2 = X_train.iloc[trainset,]
    Y_1 = Y_train.iloc[valset,]
    Y_2 = Y_train.iloc[trainset,]
    return X_2,Y_2,X_1,Y_1

X_train,Y_train,X_val,Y_val = My_Own_CF()
# On a direct les training sets de la CFvalidation

In [9]:
X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_train_reshape.columns = pd.Index(range(1,251), name='timeLag')

X_val_reshape = pd.concat([ X_val.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_val_reshape.columns = pd.Index(range(1,251), name='timeLag')

# Tout le bazar stdt, pas besoin de se le retaper à chaque fois
targets_t = torch.tensor(Y_train.T.stack().to_numpy().astype(np.float32)) # Le gros vecteur de pred 
Y_true_t = torch.tensor(Y_train.to_numpy().astype(np.float32)) # La matrice à prédire en torch
X_train_reshape_t = torch.tensor(X_train_reshape.to_numpy().astype(np.float32)) # X_train_reshape en torch

def randomA(D=250, F=10):  
    
    M = np.random.randn(D,F)
    randomStiefel = np.linalg.qr(M)[0] # Apply Gram-Schmidt algorithm to the columns of M
    
    return randomStiefel



def checkOrthonormality(A): 
    
    bool = True
    D, F = A.shape   
    Error = pd.DataFrame(A.T @ A - np.eye(F)).abs()
    
    if any(Error.unstack() > 1e-6):
        bool = False
     
    return bool

X = randomA()

def fitBeta(A):
    
    predictors = X_train_reshape @ A # the dataframe of the 10 factors created from A with the (date, stock) in index
    targets = Y_train.T.stack()
    beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets
    
    return beta.to_numpy()

checkOrthonormality(X)

def metric_train(A, beta): 
    
    if not checkOrthonormality(A):
        return -1.0    
    
    Ypred = (X_train_reshape @ A @ beta).unstack().T    # remet les prédictions sous forme de matrice 50*504. Les vecteurs de Ypred sont les prédictions !!      
    Ytrue = Y_train
    
    Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
    Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

    meanOverlap = (Ytrue * Ypred).sum().mean()

    return  meanOverlap  

def metric_val(A, beta): 
    
    if not checkOrthonormality(A):
        return -1.0    
    
    Ypred = (X_val_reshape @ A @ beta).unstack().T    # remet les prédictions sous forme de matrice 50*504. Les vecteurs de Ypred sont les prédictions !!      
    Ytrue = Y_val
    
    Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
    Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

    meanOverlap = (Ytrue * Ypred).sum().mean()

    return  meanOverlap  

#On calcule en même temps la fonction à maximiser (sortie 1) et son gradient au point A (sortie 2)
def F_et_gradient(A):
    
    A_t = torch.tensor(A.astype(np.float32),requires_grad = True) # On transforme l'entrée en tensor (on spécifie qu'on va différencier par rapport à A_t !!)
    predictors = X_train_reshape_t @ A_t # Les prédicteurs en tensor 
    beta_t = torch.inverse(predictors.T @ predictors)@predictors.T @targets_t # beta de l'équation normale en tensor
    Y_pred_t = (X_train_reshape_t @ A_t @ beta_t).resize(504,40).T #La matrice de prédiction en tensor
    f_A = torch.mean(sum(normalize(Y_pred_t,dim=0)*normalize(Y_true_t,dim=0)))
    f_A.backward() # On calcule le gradient
    A_output = -1*(A_t.grad).numpy() # Et voici le gradient en A !! C'est magique tellement c'est facile... 
    func_val = -1*f_A.detach().numpy() #la fonction de coût à minimiser évaluée en A 
    
    return func_val, A_output

C:\Users\user\AppData\Local\Temp\ipykernel_15856\838770198.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_train_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()


NameError: name 'X_val' is not defined

In [15]:
X = randomA()
bX = fitBeta(X)

In [43]:
 # Le grad search sur X_train du CF
#with open('Bch4_17_11', 'rb') as f1:
  #  XAV = pickle.load(f1)


def algo_curv_search(X_start = randomA(),rho1 = 0.000001,rho2 = 0.5,eps = 0.00001,j = 150):
    start = time.time()
    X = X_start
    i,I = 0,np.eye(250)
    f,G = F_et_gradient(X) # f is f(X) et G the gradient of f at point X 
    tau_min = 0.0000001
    tau_max = 15
    #runn_scores = []
    
    while  (np.linalg.norm(G) >= eps) and (i <= j):
        i += 1
        #if i % 5 == 0:
            #print("Nombre d'itérations:",i,"score actuel:", -f,"gradient actuel", np.linalg.norm(G))
            #print(time.ctime(time.time() - start)) # to keep track of the evolution of the algorithm
        tau_min = 0.000001
        tau_max = 15
        W = G @ X.T - X @G.T #the skew symmetric matrix of the article _ we try to stick to the notations !  Remember, we need to update these at the end of the Armijo Wolfe conditions.          
        DF_0 = np.trace(G.T @ W @ X) #the directional derivative F'(Y(0)). 
        M_inv_min = np.linalg.inv(I + tau_min/2 * W) # Useful inverse matrices which we do not want to compute twice
        M_inv_max = np.linalg.inv(I + tau_max/2 * W)
        Y_t_min = M_inv_min @ (I - tau_min/2 * W) @ X
        Y_t_max = M_inv_max @ (I - tau_max/2 * W) @ X
        f_t_max,G_t_max = F_et_gradient(Y_t_max)
        f_t_min,G_t_min = F_et_gradient(Y_t_min)
        cond_min = (f_t_min <= f + rho1 * tau_min * DF_0) # AW cond at tau_min
        cond_max = (.5 * np.trace(G_t_max.T @ M_inv_max @ W @ (X + Y_t_max)) >= rho2* DF_0) #AW cond at tau_max
        while (not cond_min) or (not cond_max) :
            if not cond_min:
                #print("Boucle d'itération:",i,"là c'est cond_min:", tau_min,tau_max) #pour check
                if tau_min < 0.01:
                    tau_min = min(10*tau_min,tau_max - 0.0000000001)
                else :
                    tau_min = min(1.05 * tau_min,tau_max - 0.000000001) 
                #tau_min = .5*(tau_min + tau_max) # dichotomy is way too violent in this case... 
                M_inv_min = np.linalg.inv(I + tau_min/2 * W)
                Y_t_min = M_inv_min @ (I - tau_min/2 * W) @ X
                f_t_min,G_t_min = F_et_gradient(Y_t_min)
                cond_min = (f_t_min <= f + rho1 * tau_min * DF_0) # AW cond at tau_min
            elif not cond_max:
                #print("Boucle d'itération:",i,"là c'est cond_max:", tau_min,tau_max) #pour check
                tau_max = max(tau_min+ 0.000000001,min(.95*tau_max,.5*(tau_max+tau_min))) # just to ensure that we always have tau_max > tau_min
                # tau_max = .5*(tau_min + tau_max) same comment
                M_inv_max = np.linalg.inv(I + tau_max/2 * W)
                Y_t_max = M_inv_max @ (I - tau_max/2 * W) @ X
                f_t_max,G_t_max = F_et_gradient(Y_t_max)
                cond_max = (.5 * np.trace(G_t_max.T @ M_inv_max @ W @ (X + Y_t_max)) >= rho2* DF_0) #AW cond at tau_max
                
        tau_f = .5*(tau_min + tau_max) # t_k satisfying both AW conditions
        Y_t_f = np.linalg.inv(I + tau_f/2 * W) @ (I - tau_f/2 * W) @ X
        f_t_f,G_t_f = F_et_gradient(Y_t_f)
        #cond_f_1 = (f_t_f <= f + rho1 * tau_f * DF_0) #for control
        #cond_f_2 = (.5 * np.trace(G_t_f.T @ np.linalg.inv(I + tau_f/2 * W) @ W @ (X + Y_t_f)) >= rho2* DF_0) #for control
        #print("Boucle d'itération:",i,"un tau qui va", tau_f, cond_f_1, cond_f_2 ) #for control 
        X = np.linalg.inv(I + tau_f/2 * W) @ (I - tau_f/2 * W) @ X
        f,G = F_et_gradient(X)
        #runn_scores.append(f)
        #if npmean(runn_scores[::-20])
        
    print("Le score final est:",-f)
    return X,-f
        
    
# renvoie n top scorers pour Xtrain
def resultatsgrad(n = 5): 
    mat=[]
    for i in range(n):
        X = randomA()
        E,sc = algo_curv_search(X_start=X)
        mat.append(E)
    return mat    


# On teste les n sur Xval; on renvoie la liste des scores sur X_val
def top_scorer(E):
    res = []
    i=1
    for mat in E:
        beta = fitBeta(mat)
        score = metric_val(mat,beta)
        print("Le score pour la matrice",i,"est",score)
        res.append(score)
        i+=1
    k = np.argmax(res)
    print("Le top scorer est le no",k)
    return E[k]
        

In [44]:
M = resultatsgrad()
top_scorer(M)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\torch\_tensor.py:760: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Le score final est: 0.1550392508506775
Le score final est: 0.15503928065299988
Le score final est: 0.15503928065299988
Le score final est: 0.15503928065299988
Le score final est: 0.15503928065299988
Le score pour la matrice 1 est 0.07336106643352831
Le score pour la matrice 2 est 0.07336514250306797
Le score pour la matrice 3 est 0.07336575636435579
Le score pour la matrice 4 est 0.07336742060891391
Le score pour la matrice 5 est 0.07336366699792572
Le top scorer est le no 3


array([[-0.04319745, -0.09150063,  0.01749819, ...,  0.02478551,
        -0.05702485,  0.05886815],
       [ 0.00387718, -0.023314  , -0.16405323, ...,  0.03729388,
        -0.14377759, -0.14592098],
       [-0.06404435,  0.08790449, -0.01489486, ...,  0.03066821,
         0.04115303,  0.01135247],
       ...,
       [ 0.05625663, -0.03724243,  0.05794762, ..., -0.0407986 ,
         0.12884013, -0.00611646],
       [-0.11360385, -0.00861126, -0.01370783, ...,  0.01199482,
         0.16568935, -0.10506742],
       [ 0.00898153,  0.03345294,  0.04480027, ...,  0.06473483,
         0.0258555 , -0.04375351]])

In [30]:
M = resultatsgrad(5)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\torch\_tensor.py:760: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Le score final est: 0.16034188866615295
Le score final est: 0.16034188866615295
Le score final est: 0.16034188866615295
Le score final est: 0.16034188866615295
Le score final est: 0.16034188866615295


In [13]:
checkOrthonormality(XAV)
        

True

In [6]:
XAV

array([[-0.02976787,  0.05537492, -0.07782243, ..., -0.05068061,
        -0.09802267,  0.0639761 ],
       [-0.06782048, -0.07839067,  0.00068759, ..., -0.04494692,
         0.05838768, -0.0069381 ],
       [-0.03383288, -0.14343419, -0.05779111, ..., -0.05000139,
        -0.0081585 ,  0.00710922],
       ...,
       [-0.01180972, -0.01559474, -0.01554548, ..., -0.15532192,
        -0.01110967,  0.02181123],
       [-0.02048849,  0.02301782,  0.10225766, ..., -0.11324442,
         0.00672905,  0.15530524],
       [-0.12801446,  0.05585329,  0.0011197 , ..., -0.05215066,
        -0.00377846, -0.06848738]])

In [14]:
locations_last_res = [31, 33, 18, 10, 23, 12, 43,  0,  6,  5]
locations = [i for i in range(50)]

In [16]:
for i in locations_last_res:
    locations.remove(i)

In [2]:
import pickle 
with open('Bch_22_11', 'rb') as f1:
    XAV = pickle.load(f1)

In [18]:
X_1 = X_train.iloc[locations,]

In [19]:
Y_1 = Y_train.iloc[locations,]

In [20]:
X_1_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
X_1_reshape.columns = pd.Index(range(1,251), name='timeLag')

#X_val_reshape = pd.concat([ X_val.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()
#X_val_reshape.columns = pd.Index(range(1,251), name='timeLag')

C:\Users\user\AppData\Local\Temp\ipykernel_15856\4063455792.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  X_1_reshape = pd.concat([ X_train.T.shift(i+1).stack(dropna=False) for i in range(250) ], 1).dropna()


In [23]:
predictors = X_1_reshape @ XAV # the dataframe of the 10 factors created from A with the (date, stock) in index
predictors.shape

(25200, 10)

In [21]:
targets = Y_1.T.stack()

In [21]:
beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets

ValueError: matrices are not aligned